# Strats Baseline Forecasting 

redo with new dataset just to be sure

In [10]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [11]:
! export TF_CPP_MIN_LOG_LEVEL=2

## Hardware check

In [12]:
# gpu check
!nvidia-smi

Thu Nov 30 18:22:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:3A:00.0 Off |                    0 |
| N/A   31C    P0    41W / 300W |     15MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:3B:00.0 Off |                    0 |
| N/A   

In [13]:
# check number of cores
import multiprocessing

cores = multiprocessing.cpu_count() 
cores

80

## Environment Prep

In [14]:
cd /pfs/data5/home/hd/hd_hd/hd_nf283/MA_Thesis

/pfs/data5/home/hd/hd_hd/hd_nf283/MA_Thesis


In [15]:
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Lambda
# from tensorflow.keras.models import Model
from tensorflow.keras import models
import pickle
import numpy as np
from tqdm import tqdm
tqdm.pandas()
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import Callback, EarlyStopping
import pandas as pd
import json
from torch.utils.data import Dataset
from transformers import AutoTokenizer, pipeline, AutoModel
import resources.smart_cond as sc
import gc
# from google.colab import files

In [16]:
import tensorflow as tf
print(tf. __version__)

pickle.format_version

2.12.0


'4.0'

In [17]:
data_path = 'Data/sepsis_removed_0.pkl'
pkl = pickle.load(open(data_path, 'rb'))
data = pkl[0]
oc = pkl[1]
train_ind = pkl[2]
valid_ind = pkl[3]
test_ind = pkl[4]
del pkl

In [18]:
data.loc[data['variable'] == 'Text', 'value'] = 1
data

,ts_ind,hour,variable,value,TABLE,mean,std
0,10223,467.816667,Text,1,noteevents,1.000000,1.000000
1,18407,28.016667,Text,1,noteevents,1.000000,1.000000
2,40300,155.166667,Text,1,noteevents,1.000000,1.000000
3,23747,52.383333,Text,1,noteevents,1.000000,1.000000
4,2357,73.133333,Text,1,noteevents,1.000000,1.000000
...,...,...,...,...,...,...,...
82886223,57281,20.400000,MBP,0.195381,chart,78.552377,17.645628
82886224,57281,20.400000,O2 Saturation,-0.678068,chart,96.820961,4.160290
82886225,57281,20.400000,RR,0.179866,chart,26.278501,15.130729
82886226,57281,20.400000,SBP,-0.404061,chart,120.239648,25.341836


In [19]:
pred_window = 2 # hours
obs_windows = range(20, 124, 4)
# Remove test patients.
data = data.merge(oc[['ts_ind', 'SUBJECT_ID']], on='ts_ind', how='left')
test_sub = oc.loc[oc.ts_ind.isin(test_ind)].SUBJECT_ID.unique()
data = data.loc[~data.SUBJECT_ID.isin(test_sub)]
oc = oc.loc[~oc.SUBJECT_ID.isin(test_sub)]
data.drop(columns=['SUBJECT_ID', 'TABLE'], inplace=True)
# Fix age.
data.loc[(data.variable=='Age')&(data.value>200), 'value'] = 91.4
# data[data.variable=='Age'][data.value>200]['value'] = 91.4

In [20]:
# Get static data with mean fill and missingness indicator.
static_varis = ['Age', 'Gender']
ii = data.variable.isin(static_varis)
static_data = data.loc[ii]
data = data.loc[~ii]
def inv_list(l, start=0):
    d = {}
    for i in range(len(l)):
        d[l[i]] = i+start
    return d
static_var_to_ind = inv_list(static_varis)
D = len(static_varis)
N = data.ts_ind.max()+1
demo = np.zeros((N, D))
for row in tqdm(static_data.itertuples()):
    demo[row.ts_ind, static_var_to_ind[row.variable]] = row.value
# Normalize static data.
means = demo.mean(axis=0, keepdims=True)
stds = demo.std(axis=0, keepdims=True)
stds = (stds==0)*1 + (stds!=0)*stds
demo = (demo-means)/stds
# Get variable indices.
varis = sorted(list(set(data.variable)))
V = len(varis)
var_to_ind = inv_list(varis, start=1)
data['vind'] = data.variable.map(var_to_ind)
data = data[['ts_ind', 'vind', 'hour', 'value']].sort_values(by=['ts_ind', 'vind', 'hour'])
# Find max_len.
fore_max_len = 880
# Get forecast inputs and outputs.
fore_times_ip = []
fore_values_ip = []
fore_varis_ip = []
fore_op = []
fore_inds = []
def f(x):
    mask = [0 for i in range(V)]
    values = [0 for i in range(V)]
    for vv in x:
        v = int(vv[0])-1
        mask[v] = 1
        values[v] = vv[1]
    return values+mask
def pad(x):
    return x+[0]*(fore_max_len-len(x))
for w in tqdm(obs_windows):
    pred_data = data.loc[(data.hour>=w)&(data.hour<=w+pred_window)]
    pred_data = pred_data.groupby(['ts_ind', 'vind']).agg({'value':'first'}).reset_index()
    pred_data['vind_value'] = pred_data[['vind', 'value']].values.tolist()
    pred_data = pred_data.groupby('ts_ind').agg({'vind_value':list}).reset_index()
    pred_data['vind_value'] = pred_data['vind_value'].apply(f)    
    obs_data = data.loc[(data.hour<w)&(data.hour>=w-24)]
    obs_data = obs_data.loc[obs_data.ts_ind.isin(pred_data.ts_ind)]
    obs_data = obs_data.groupby('ts_ind').head(fore_max_len)
    obs_data = obs_data.groupby('ts_ind').agg({'vind':list, 'hour':list, 'value':list}).reset_index()
    obs_data = obs_data.merge(pred_data, on='ts_ind')
    for col in ['vind', 'hour', 'value']:
        obs_data[col] = obs_data[col].apply(pad)
    fore_op.append(np.array(list(obs_data.vind_value)))
    fore_inds.append(np.array(list(obs_data.ts_ind)))
    fore_times_ip.append(np.array(list(obs_data.hour)))
    fore_values_ip.append(np.array(list(obs_data.value)))
    fore_varis_ip.append(np.array(list(obs_data.vind)))
del data
fore_times_ip = np.concatenate(fore_times_ip, axis=0)
fore_values_ip = np.concatenate(fore_values_ip, axis=0)
fore_varis_ip = np.concatenate(fore_varis_ip, axis=0)
fore_op = np.concatenate(fore_op, axis=0)
fore_inds = np.concatenate(fore_inds, axis=0)
fore_demo = demo[fore_inds]
# Get train and valid ts_ind for forecast task.
train_sub = oc.loc[oc.ts_ind.isin(train_ind)].SUBJECT_ID.unique()
valid_sub = oc.loc[oc.ts_ind.isin(valid_ind)].SUBJECT_ID.unique()
rem_sub = oc.loc[~oc.SUBJECT_ID.isin(np.concatenate((train_ind, valid_ind)))].SUBJECT_ID.unique()
bp = int(0.8*len(rem_sub))
train_sub = np.concatenate((train_sub, rem_sub[:bp]))
valid_sub = np.concatenate((valid_sub, rem_sub[bp:]))
train_ind = oc.loc[oc.SUBJECT_ID.isin(train_sub)].ts_ind.unique() # Add remaining ts_ind s of train subjects.
valid_ind = oc.loc[oc.SUBJECT_ID.isin(valid_sub)].ts_ind.unique() # Add remaining ts_ind s of train subjects.
# Generate 3 sets of inputs and outputs.
train_ind = np.argwhere(np.in1d(fore_inds, train_ind)).flatten()
valid_ind = np.argwhere(np.in1d(fore_inds, valid_ind)).flatten()
fore_train_ip = [ip[train_ind] for ip in [fore_demo, fore_times_ip, fore_values_ip, fore_varis_ip]]
fore_valid_ip = [ip[valid_ind] for ip in [fore_demo, fore_times_ip, fore_values_ip, fore_varis_ip]]
del fore_times_ip, fore_values_ip, fore_varis_ip, demo, fore_demo
fore_train_op = fore_op[train_ind]
fore_valid_op = fore_op[valid_ind]
del fore_op

91626it [00:00, 826485.08it/s]
100%|██████████| 26/26 [04:56<00:00, 11.39s/it]


In [21]:
def get_res(y_true, y_pred):
    precision, recall, thresholds = precision_recall_curve(y_true, y_pred)
    pr_auc = auc(recall, precision)
    minrp = np.minimum(precision, recall).max()
    roc_auc = roc_auc_score(y_true, y_pred)
    return [roc_auc, pr_auc, minrp]

def forecast_loss(y_true, y_pred):
    return K.sum(y_true[:,V:]*(y_true[:,:V]-y_pred)**2, axis=-1)

def get_min_loss(weight):
    def min_loss(y_true, y_pred):
        return weight*y_pred
    return min_loss

class CustomCallback(Callback):
    def __init__(self, validation_data, batch_size):
        self.val_x, self.val_y = validation_data
        self.batch_size = batch_size
        super(Callback, self).__init__()

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.val_x, verbose=0, batch_size=self.batch_size)
        if type(y_pred)==type([]):
            y_pred = y_pred[0]
        precision, recall, thresholds = precision_recall_curve(self.val_y, y_pred)
        pr_auc = auc(recall, precision)
        roc_auc = roc_auc_score(self.val_y, y_pred)
        logs['custom_metric'] = pr_auc + roc_auc
        print ('val_aucs:', pr_auc, roc_auc)

In [22]:
import tensorflow as tf
import numpy as np
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Embedding, Activation, Dropout, Softmax, Layer, InputSpec, Input, Dense, Lambda, TimeDistributed, Concatenate, Add
from tensorflow.keras import initializers, regularizers, constraints, Model
from tensorflow.python.keras.utils import tf_utils
from tensorflow.python.ops import array_ops
from tensorflow import nn

    
class CVE(Layer):
    def __init__(self, hid_units, output_dim):
        self.hid_units = hid_units
        self.output_dim = output_dim
        super(CVE, self).__init__()
        
    def build(self, input_shape): 
        self.W1 = self.add_weight(name='CVE_W1',
                            shape=(1, self.hid_units),
                            initializer='glorot_uniform',
                            trainable=True)
        self.b1 = self.add_weight(name='CVE_b1',
                            shape=(self.hid_units,),
                            initializer='zeros',
                            trainable=True)
        self.W2 = self.add_weight(name='CVE_W2',
                            shape=(self.hid_units, self.output_dim),
                            initializer='glorot_uniform',
                            trainable=True)
        super(CVE, self).build(input_shape)
        
    def call(self, x):
        x = K.expand_dims(x, axis=-1)
        x = K.dot(K.tanh(K.bias_add(K.dot(x, self.W1), self.b1)), self.W2)
        return x
        
    def compute_output_shape(self, input_shape):
        return input_shape + (self.output_dim,)
    
    
class Attention(Layer):
    
    def __init__(self, hid_dim):
        self.hid_dim = hid_dim
        super(Attention, self).__init__()

    def build(self, input_shape):
        d = input_shape.as_list()[-1]
        self.W = self.add_weight(shape=(d, self.hid_dim), name='Att_W',
                                 initializer='glorot_uniform',
                                 trainable=True)
        self.b = self.add_weight(shape=(self.hid_dim,), name='Att_b',
                                 initializer='zeros',
                                 trainable=True)
        self.u = self.add_weight(shape=(self.hid_dim,1), name='Att_u',
                                 initializer='glorot_uniform',
                                 trainable=True)
        super(Attention, self).build(input_shape)
        
    def call(self, x, mask, mask_value=-1e30):
        attn_weights = K.dot(K.tanh(K.bias_add(K.dot(x,self.W), self.b)), self.u)
        mask = K.expand_dims(mask, axis=-1)
        attn_weights = mask*attn_weights + (1-mask)*mask_value
        attn_weights = K.softmax(attn_weights, axis=-2)
        return attn_weights
        
    def compute_output_shape(self, input_shape):
        return input_shape[:-1] + (1,)
    
    
class Transformer(Layer):
    
    def __init__(self, N=2, h=8, dk=None, dv=None, dff=None, dropout=0):
        self.N, self.h, self.dk, self.dv, self.dff, self.dropout = N, h, dk, dv, dff, dropout
        self.epsilon = K.epsilon() * K.epsilon()
        super(Transformer, self).__init__()

    def build(self, input_shape):
        d = input_shape.as_list()[-1]
        if self.dk==None:
            self.dk = d//self.h
        if self.dv==None:
            self.dv = d//self.h
        if self.dff==None:
            self.dff = 2*d
        self.Wq = self.add_weight(shape=(self.N, self.h, d, self.dk), name='Wq',
                                 initializer='glorot_uniform', trainable=True)
        self.Wk = self.add_weight(shape=(self.N, self.h, d, self.dk), name='Wk',
                                 initializer='glorot_uniform', trainable=True)
        self.Wv = self.add_weight(shape=(self.N, self.h, d, self.dv), name='Wv',
                                 initializer='glorot_uniform', trainable=True)
        self.Wo = self.add_weight(shape=(self.N, self.dv*self.h, d), name='Wo',
                                 initializer='glorot_uniform', trainable=True)
        self.W1 = self.add_weight(shape=(self.N, d, self.dff), name='W1',
                                 initializer='glorot_uniform', trainable=True)
        self.b1 = self.add_weight(shape=(self.N, self.dff), name='b1',
                                 initializer='zeros', trainable=True)
        self.W2 = self.add_weight(shape=(self.N, self.dff, d), name='W2',
                                 initializer='glorot_uniform', trainable=True)
        self.b2 = self.add_weight(shape=(self.N, d), name='b2',
                                 initializer='zeros', trainable=True)
        self.gamma = self.add_weight(shape=(2*self.N,), name='gamma',
                                 initializer='ones', trainable=True)
        self.beta = self.add_weight(shape=(2*self.N,), name='beta',
                                 initializer='zeros', trainable=True)
        super(Transformer, self).build(input_shape)
        
    def call(self, x, mask, mask_value=-1e-30):
        mask = K.expand_dims(mask, axis=-2)
        for i in range(self.N):
            # MHA
            mha_ops = []
            for j in range(self.h):
                q = K.dot(x, self.Wq[i,j,:,:])
                k = K.permute_dimensions(K.dot(x, self.Wk[i,j,:,:]), (0,2,1))
                v = K.dot(x, self.Wv[i,j,:,:])
                A = K.batch_dot(q,k)
                # Mask unobserved steps.
                A = mask*A + (1-mask)*mask_value
                # Mask for attention dropout.
                def dropped_A():
                    dp_mask = K.cast((K.random_uniform(shape=array_ops.shape(A))>=self.dropout), K.floatx())
                    return A*dp_mask + (1-dp_mask)*mask_value
                A = sc.smart_cond(K.learning_phase(), dropped_A, lambda: array_ops.identity(A))
                A = K.softmax(A, axis=-1)
                mha_ops.append(K.batch_dot(A,v))
            conc = K.concatenate(mha_ops, axis=-1)
            proj = K.dot(conc, self.Wo[i,:,:])
            # Dropout.
            proj = sc.smart_cond(K.learning_phase(), lambda: array_ops.identity(nn.dropout(proj, rate=self.dropout)),\
                                       lambda: array_ops.identity(proj))
            # Add & LN
            x = x+proj
            mean = K.mean(x, axis=-1, keepdims=True)
            variance = K.mean(K.square(x - mean), axis=-1, keepdims=True)
            std = K.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x*self.gamma[2*i] + self.beta[2*i]
            # FFN
            ffn_op = K.bias_add(K.dot(K.relu(K.bias_add(K.dot(x, self.W1[i,:,:]), self.b1[i,:])), 
                           self.W2[i,:,:]), self.b2[i,:,])
            # Dropout.
            ffn_op = sc.smart_cond(K.learning_phase(), lambda: array_ops.identity(nn.dropout(ffn_op, rate=self.dropout)),\
                                       lambda: array_ops.identity(ffn_op))
            # Add & LN
            x = x+ffn_op
            mean = K.mean(x, axis=-1, keepdims=True)
            variance = K.mean(K.square(x - mean), axis=-1, keepdims=True)
            std = K.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x*self.gamma[2*i+1] + self.beta[2*i+1]            
        return x
        
    def compute_output_shape(self, input_shape):
        return input_shape


def build_strats(D, max_len, V, d, N, he, dropout, forecast=False):
    demo = Input(shape=(D,))
    demo_enc = Dense(2*d, activation='tanh')(demo)
    demo_enc = Dense(d, activation='tanh')(demo_enc)
    varis = Input(shape=(max_len,))
    values = Input(shape=(max_len,))
    times = Input(shape=(max_len,))
    varis_emb = Embedding(V+1, d)(varis)
    cve_units = int(np.sqrt(d))
    values_emb = CVE(cve_units, d)(values)
    times_emb = CVE(cve_units, d)(times)
    comb_emb = Add()([varis_emb, values_emb, times_emb]) # b, L, d
#     demo_enc = Lambda(lambda x:K.expand_dims(x, axis=-2))(demo_enc) # b, 1, d
#     comb_emb = Concatenate(axis=-2)([demo_enc, comb_emb]) # b, L+1, d
    mask = Lambda(lambda x:K.clip(x,0,1))(varis) # b, L
#     mask = Lambda(lambda x:K.concatenate((K.ones_like(x)[:,0:1], x), axis=-1))(mask) # b, L+1
    cont_emb = Transformer(N, he, dk=None, dv=None, dff=None, dropout=dropout)(comb_emb, mask=mask)
    attn_weights = Attention(2*d)(cont_emb, mask=mask)
    fused_emb = Lambda(lambda x:K.sum(x[0]*x[1], axis=-2))([cont_emb, attn_weights])
    conc = Concatenate(axis=-1)([fused_emb, demo_enc])
    fore_op = Dense(V)(conc)
    op = Dense(1, activation='sigmoid')(fore_op)
    model = Model([demo, times, values, varis], op)
    if forecast:
        fore_model = Model([demo, times, values, varis], fore_op)
        return [model, fore_model]
    return model

# To tune:
# 1. Transformer parameters. (N, h, dropout)
# 2. Normalization

In [23]:
# # lr, batch_size, samples_per_epoch, patience = 0.0005, 32, 102400, 5
# lr, batch_size, samples_per_epoch, patience = 0.0005, 32, 102400, 10
# d, N, he, dropout = 50, 2, 4, 0.2
# model, fore_model =  build_strats(D, fore_max_len, V, d, N, he, dropout, forecast=True)
# print (fore_model.summary())
# fore_model.compile(loss=forecast_loss, optimizer=Adam(lr))

# # Pretrain fore_model.
# best_val_loss = np.inf
# N_fore = len(fore_train_op)
# fore_savepath = 'Baseline/models/forecasting/strats_baseline_forecasting_patience_10.h5'

# train_losses = []
# val_losses = []

# for e in range(1000):
#     e_indices = np.random.choice(range(N_fore), size=samples_per_epoch, replace=False)
#     e_loss = 0
#     pbar = tqdm(range(0, len(e_indices), batch_size))
#     for start in pbar:
#         ind = e_indices[start:start+batch_size]
#         e_loss += fore_model.train_on_batch([ip[ind] for ip in fore_train_ip], fore_train_op[ind])
#         pbar.set_description('%f'%(e_loss/(start+1)))
#     val_loss = fore_model.evaluate(fore_valid_ip, fore_valid_op, batch_size=batch_size, verbose=1)
    
#     train_losses.append(e_loss*batch_size/samples_per_epoch)
#     val_losses.append(val_loss)
    
#     print ('Epoch', e, 'loss', e_loss*batch_size/samples_per_epoch, 'val loss', val_loss)
    
#     print(train_losses)
#     print(val_losses)
    
#     if val_loss < best_val_loss:
#         best_val_loss = val_loss
#         fore_model.save_weights(fore_savepath)
#         best_epoch = e
#     if (e-best_epoch)>patience:
#         break

In [24]:
val_losses = [7.717712879180908, 6.955950736999512, 6.704439640045166, 6.528388023376465, 6.3274617195129395, 6.1996750831604, 6.113377094268799, 6.011967658996582, 5.910389423370361, 5.87477970123291, 5.860624313354492, 5.7814507484436035, 5.771292686462402, 5.716648101806641, 5.757233619689941, 5.679626941680908, 5.713437080383301, 5.611827850341797, 5.663970947265625, 5.621941566467285, 5.755812644958496, 5.623030662536621, 5.574612617492676, 5.539113998413086, 5.730890274047852, 5.500331401824951, 5.504433631896973, 5.448913097381592, 5.507577896118164, 5.510786056518555, 5.459959983825684, 5.4007673263549805, 5.396493434906006, 5.44691276550293, 5.382143497467041, 5.412578582763672, 5.4137701988220215, 5.555471897125244]
min(val_losses)

5.382143497467041

In [ ]:
len(val_losses)

38

In [ ]:
train_losses = [8.99688381895423, 7.447008701860905, 6.819614188373089, 6.756220295354724, 6.3309409860521555, 6.210726347230374, 6.219333339333534, 6.225389459840954, 5.978656517714262, 5.980397833921015, 5.72186337761581, 5.895540365539492, 5.849212531298399, 5.690261988639832, 5.624563320577145, 5.679155041836202, 5.586134704425931, 5.654708657674491, 5.715709848292172, 5.5188806889951225, 5.497861228436232, 5.571464146263898, 5.4665602750703695, 5.551240729577839, 5.368751032687723, 5.420417926572263, 5.53939581360668, 5.477566199973226, 5.522608667239547, 5.405717139951885, 5.456880302615463, 5.49130221568048, 5.47696663826704, 5.463503729626536, 5.355901707224548, 5.425206321999431, 5.4836160293594, 5.3817997147515415]
val_losses = [7.717712879180908, 6.955950736999512, 6.704439640045166, 6.528388023376465, 6.3274617195129395, 6.1996750831604, 6.113377094268799, 6.011967658996582, 5.910389423370361, 5.87477970123291, 5.860624313354492, 5.7814507484436035, 5.771292686462402, 5.716648101806641, 5.757233619689941, 5.679626941680908, 5.713437080383301, 5.611827850341797, 5.663970947265625, 5.621941566467285, 5.755812644958496, 5.623030662536621, 5.574612617492676, 5.539113998413086, 5.730890274047852, 5.500331401824951, 5.504433631896973, 5.448913097381592, 5.507577896118164, 5.510786056518555, 5.459959983825684, 5.4007673263549805, 5.396493434906006, 5.44691276550293, 5.382143497467041, 5.412578582763672, 5.4137701988220215, 5.555471897125244]

lr, batch_size, samples_per_epoch, patience = 0.0005, 32, 102400, 10
d, N, he, dropout = 50, 2, 4, 0.2
model, fore_model =  build_strats(D, fore_max_len, V, d, N, he, dropout, forecast=True)
print (fore_model.summary())
fore_model.compile(loss=forecast_loss, optimizer=Adam(lr))
fore_savepath = 'Baseline/models/forecasting/strats_baseline_forecasting_patience_10.h5'
fore_model.load_weights(fore_savepath)


# Pretrain fore_model.
best_val_loss = 5.382143497467041
N_fore = len(fore_train_op)

# train_losses = []
# val_losses = []

best_epoch = 34

for e in range(38, 1000):
    e_indices = np.random.choice(range(N_fore), size=samples_per_epoch, replace=False)
    e_loss = 0
    pbar = tqdm(range(0, len(e_indices), batch_size))
    for start in pbar:
        ind = e_indices[start:start+batch_size]
        e_loss += fore_model.train_on_batch([ip[ind] for ip in fore_train_ip], fore_train_op[ind])
        pbar.set_description('%f'%(e_loss/(start+1)))
    val_loss = fore_model.evaluate(fore_valid_ip, fore_valid_op, batch_size=batch_size, verbose=1)
    
    train_losses.append(e_loss*batch_size/samples_per_epoch)
    val_losses.append(val_loss)
    
    print ('Epoch', e, 'loss', e_loss*batch_size/samples_per_epoch, 'val loss', val_loss)
    
    print(train_losses)
    print(val_losses)
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        fore_model.save_weights(fore_savepath)
        best_epoch = e
    if (e-best_epoch)>patience:
        break

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 880)]        0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 880)]        0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 880)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 880, 50)      6750        ['input_2[0][0]']                
                                                                                            

0.165991: 100%|██████████| 3200/3200 [02:41<00:00, 19.80it/s]


4276/4276 [==============================] - 69s 16ms/step - loss: 5.4334
Epoch 38 loss 5.310110193789005 val loss 5.43337869644165
[8.99688381895423, 7.447008701860905, 6.819614188373089, 6.756220295354724, 6.3309409860521555, 6.210726347230374, 6.219333339333534, 6.225389459840954, 5.978656517714262, 5.980397833921015, 5.72186337761581, 5.895540365539492, 5.849212531298399, 5.690261988639832, 5.624563320577145, 5.679155041836202, 5.586134704425931, 5.654708657674491, 5.715709848292172, 5.5188806889951225, 5.497861228436232, 5.571464146263898, 5.4665602750703695, 5.551240729577839, 5.368751032687723, 5.420417926572263, 5.53939581360668, 5.477566199973226, 5.522608667239547, 5.405717139951885, 5.456880302615463, 5.49130221568048, 5.47696663826704, 5.463503729626536, 5.355901707224548, 5.425206321999431, 5.4836160293594, 5.3817997147515415, 5.310110193789005]
[7.717712879180908, 6.955950736999512, 6.704439640045166, 6.528388023376465, 6.3274617195129395, 6.1996750831604, 6.1133770942687

0.165493: 100%|██████████| 3200/3200 [02:25<00:00, 22.03it/s]


4276/4276 [==============================] - 69s 16ms/step - loss: 5.4917
Epoch 39 loss 5.2941837514936925 val loss 5.491706371307373
[8.99688381895423, 7.447008701860905, 6.819614188373089, 6.756220295354724, 6.3309409860521555, 6.210726347230374, 6.219333339333534, 6.225389459840954, 5.978656517714262, 5.980397833921015, 5.72186337761581, 5.895540365539492, 5.849212531298399, 5.690261988639832, 5.624563320577145, 5.679155041836202, 5.586134704425931, 5.654708657674491, 5.715709848292172, 5.5188806889951225, 5.497861228436232, 5.571464146263898, 5.4665602750703695, 5.551240729577839, 5.368751032687723, 5.420417926572263, 5.53939581360668, 5.477566199973226, 5.522608667239547, 5.405717139951885, 5.456880302615463, 5.49130221568048, 5.47696663826704, 5.463503729626536, 5.355901707224548, 5.425206321999431, 5.4836160293594, 5.3817997147515415, 5.310110193789005, 5.2941837514936925]
[7.717712879180908, 6.955950736999512, 6.704439640045166, 6.528388023376465, 6.3274617195129395, 6.19967508

...

4276/4276 [==============================] - 68s 16ms/step - loss: 5.2692
Epoch 78 loss 5.1620757971704005 val loss 5.269227504730225
[8.99688381895423, 7.447008701860905, 6.819614188373089, 6.756220295354724, 6.3309409860521555, 6.210726347230374, 6.219333339333534, 6.225389459840954, 5.978656517714262, 5.980397833921015, 5.72186337761581, 5.895540365539492, 5.849212531298399, 5.690261988639832, 5.624563320577145, 5.679155041836202, 5.586134704425931, 5.654708657674491, 5.715709848292172, 5.5188806889951225, 5.497861228436232, 5.571464146263898, 5.4665602750703695, 5.551240729577839, 5.368751032687723, 5.420417926572263, 5.53939581360668, 5.477566199973226, 5.522608667239547, 5.405717139951885, 5.456880302615463, 5.49130221568048, 5.47696663826704, 5.463503729626536, 5.355901707224548, 5.425206321999431, 5.4836160293594, 5.3817997147515415, 5.310110193789005, 5.2941837514936925, 5.384898569844663, 5.439891175143421, 5.379424596503377, 5.37850166361779, 5.273312831632793, 5.39402168940

0.160088: 100%|██████████| 3200/3200 [02:24<00:00, 22.08it/s]


4276/4276 [==============================] - 68s 16ms/step - loss: 5.2378
Epoch 79 loss 5.121279909200966 val loss 5.23778772354126
[8.99688381895423, 7.447008701860905, 6.819614188373089, 6.756220295354724, 6.3309409860521555, 6.210726347230374, 6.219333339333534, 6.225389459840954, 5.978656517714262, 5.980397833921015, 5.72186337761581, 5.895540365539492, 5.849212531298399, 5.690261988639832, 5.624563320577145, 5.679155041836202, 5.586134704425931, 5.654708657674491, 5.715709848292172, 5.5188806889951225, 5.497861228436232, 5.571464146263898, 5.4665602750703695, 5.551240729577839, 5.368751032687723, 5.420417926572263, 5.53939581360668, 5.477566199973226, 5.522608667239547, 5.405717139951885, 5.456880302615463, 5.49130221568048, 5.47696663826704, 5.463503729626536, 5.355901707224548, 5.425206321999431, 5.4836160293594, 5.3817997147515415, 5.310110193789005, 5.2941837514936925, 5.384898569844663, 5.439891175143421, 5.379424596503377, 5.37850166361779, 5.273312831632793, 5.3940216894075

0.158179: 100%|██████████| 3200/3200 [02:25<00:00, 21.92it/s]


4276/4276 [==============================] - 68s 16ms/step - loss: 5.2837
Epoch 80 loss 5.060195054039359 val loss 5.283673286437988
[8.99688381895423, 7.447008701860905, 6.819614188373089, 6.756220295354724, 6.3309409860521555, 6.210726347230374, 6.219333339333534, 6.225389459840954, 5.978656517714262, 5.980397833921015, 5.72186337761581, 5.895540365539492, 5.849212531298399, 5.690261988639832, 5.624563320577145, 5.679155041836202, 5.586134704425931, 5.654708657674491, 5.715709848292172, 5.5188806889951225, 5.497861228436232, 5.571464146263898, 5.4665602750703695, 5.551240729577839, 5.368751032687723, 5.420417926572263, 5.53939581360668, 5.477566199973226, 5.522608667239547, 5.405717139951885, 5.456880302615463, 5.49130221568048, 5.47696663826704, 5.463503729626536, 5.355901707224548, 5.425206321999431, 5.4836160293594, 5.3817997147515415, 5.310110193789005, 5.2941837514936925, 5.384898569844663, 5.439891175143421, 5.379424596503377, 5.37850166361779, 5.273312831632793, 5.394021689407

In [ ]:
# train_losses = [8.99688381895423, 7.447008701860905, 6.819614188373089, 6.756220295354724, 6.3309409860521555, 6.210726347230374, 6.219333339333534, 6.225389459840954, 5.978656517714262, 5.980397833921015, 5.72186337761581, 5.895540365539492, 5.849212531298399, 5.690261988639832, 5.624563320577145, 5.679155041836202, 5.586134704425931, 5.654708657674491, 5.715709848292172, 5.5188806889951225, 5.497861228436232, 5.571464146263898, 5.4665602750703695, 5.551240729577839, 5.368751032687723, 5.420417926572263, 5.53939581360668, 5.477566199973226, 5.522608667239547, 5.405717139951885, 5.456880302615463, 5.49130221568048, 5.47696663826704, 5.463503729626536, 5.355901707224548, 5.425206321999431, 5.4836160293594, 5.3817997147515415]
# val_losses = [7.717712879180908, 6.955950736999512, 6.704439640045166, 6.528388023376465, 6.3274617195129395, 6.1996750831604, 6.113377094268799, 6.011967658996582, 5.910389423370361, 5.87477970123291, 5.860624313354492, 5.7814507484436035, 5.771292686462402, 5.716648101806641, 5.757233619689941, 5.679626941680908, 5.713437080383301, 5.611827850341797, 5.663970947265625, 5.621941566467285, 5.755812644958496, 5.623030662536621, 5.574612617492676, 5.539113998413086, 5.730890274047852, 5.500331401824951, 5.504433631896973, 5.448913097381592, 5.507577896118164, 5.510786056518555, 5.459959983825684, 5.4007673263549805, 5.396493434906006, 5.44691276550293, 5.382143497467041, 5.412578582763672, 5.4137701988220215, 5.555471897125244]

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np

# train_losses = [8.957182633876801, 7.444346153885126, 6.718132002651691, 6.606167373768986, 6.334112568087876, 6.222601437494159, 6.099050151444971, 6.009203886091709, 5.77085709810257, 5.949807133153081, 5.8031801933795215, 5.635595956072211, 5.798708248250186, 5.686658313274384, 5.617710028290748, 5.546603025831282, 5.4439452893659475, 5.569199087247252, 5.440258623324334, 5.6310957011580465, 5.472006217092275, 5.4239358650147915, 5.410358652472496, 5.457051942534744, 5.244602517709136, 5.531149811521172, 5.505790843628347, 5.510934510603547, 5.33967715986073]
# plt.plot(train_losses)
# plt.title('Training loss over epochs (baseline model)')
# plt.xlabel('epochs')
# plt.ylabel('training loss')
# plt.savefig('plots/train_losses.png')
# plt.show()

In [ ]:
# val_losses = [7.574024677276611, 6.8714728355407715, 6.6017351150512695, 6.405410289764404, 6.308310508728027, 6.093061923980713, 5.992845058441162, 6.107646465301514, 5.939321994781494, 5.784503936767578, 5.81207275390625, 5.778130054473877, 5.717212677001953, 5.657232284545898, 5.574744701385498, 5.6003875732421875, 5.6966986656188965, 5.5241217613220215, 5.555541515350342, 5.584465503692627, 5.486431121826172, 5.501025199890137, 5.439569473266602, 5.72035551071167, 5.5537919998168945, 5.444085597991943, 5.448609828948975, 5.465628147125244, 5.454106330871582]
# plt.plot(val_losses)
# plt.title('Validation loss over epochs (baseline model)')
# plt.xlabel('epochs')
# plt.ylabel('validation loss')
# plt.savefig('plots/validation_losses.png')
# plt.show()

In [30]:
print('haha')

haha
